# **Ubrzani NNLS algoritam (FNNLS)**

<br>
<br>


Vreme potrebno za kreiranje NNLS modela je obično mnogo duže nego u slučaju kada konstruišemo model bez ograničenja. U određenim primenama efikasnost je od ključne važnosti stoga se pribegava aproksimativnim metodama.

<br>

**Prva modifikacija** standardnog NNLS algoritma se sastoji u tome kako računamo gradijentni vektor. Umesto $$w=A^T(b-Ax)$$ računaćemo ga kao $$w = (A^Tb) - (A^TA)x$$ Ovo doprinosi 
- Složenosti izračunavanja: Za matrice velikih dimenzija, jeftinije je izračunati $A^TA$ i $A^Tb$ nego direktno sprovoditi operacije sa matricom $A$ i vektorom $b$, posebno u slučaju kada je $A^TA$ znatno manja od $A$
- Numeričkoj stabilnosti

<br>
<br>

**Druga modifikacija** se odnosi na to kako računamo vektor s. Umesto $s^P = [(A^P)^TA^P]^{-1}(A^P)^Tb$, vektor s ćemo računati kao $s^P = [(A^TA)^P]^{-1}(A^Tb)^P$, gde je $(A^TA)^P$ podmatrica od $A^TA$ koja sadrži samo redove i kolone koje odgovaraju skupu $P$ (može se proveriti da su ova dva vektora jednaka). U slučaju kada matrica A ima dosta više redova nego kolona, ovo značajno doprinosi ubrzanju.

FNNLS algoritam ažurira skup P i vodi evidenciju o ne-nula elementima rešenja (support vektor) u trenutnoj iteraciji. Često je moguće proceniti support vektor što može dodatno da ubrza ceo proces.

  **FNNLS:**

In [6]:
import numpy as np
def fnnls(A,b,epsilon,P_initial = np.zeros(0, dtype=int)): # Support vektor kao opcioni parametar
    m,n = A.shape 
    ATA = A.T.dot(A) 
    ATb = A.T.dot(b)
    max_repetitions = 5 # Broj koliko puta ćemo dopustiti da se skup P ponovi 
    
    P = np.zeros(n,dtype=bool) # Boolean indexing za dodatnu efikasnost
    P[P_initial] = True 
    x = np.zeros(n)
    w = ATb - (ATA).dot(x) # Gradijentni vektor računamo ovako
    s = np.zeros(n)
    no_update = 0 
    
    if P_initial.shape[0] != 0:

        s[P] = np.linalg.inv((ATA)[P][:,P]).dot((ATb)[P])
        x = s.clip(min=0)
    
    
    while (not np.all(P)) and np.max(w[~P]) > epsilon:
        current_P = P.copy()
        
        P[np.argmax(w * ~P)] = True
        
        s[P] = np.linalg.inv((ATA)[P][:,P]).dot((ATb)[P]) # s računamo ovako
        
        if np.any(P) and np.min(s[P]) <= epsilon:
            s,x,P = inner_loop(ATA,ATb,s,x,P,epsilon)
    
        x = s.copy()
        
        w=ATb - (ATA).dot(x)
        
        if(np.all(current_P == P)):
            no_update +=1
        else:
            no_update = 0
            
        if no_update >= max_repetitions: # Ukoliko smo dostigli limit izlazimo iz petlje
            break
            
    res = np.linalg.norm(b-A.dot(x)) # ||b-Ax||
    
    return [x,res]

def inner_loop(ATA,ATb,s,x,P,epsilon): # U ovu petlju se jako retko ulazi u praksi,
    # u proseku u 1.41% slucajeva
    q = P * (s<=epsilon)
    alpha = np.min(x[q]/ (x[q]-s[q]))
    
    x = x+alpha*(s-x)
    
    P[x<=epsilon] = False
    
    s[P] = np.linalg.inv((ATA)[P][:,P]).dot((ATb)[P])
    
    s[~P] = 0
    
    return s,x,P

**Primer:**

In [7]:
from time import time

In [12]:
np.random.seed(7)
A = np.abs(np.random.rand(100,100))
b = np.abs(np.random.rand(100))
start = time()
x,res= fnnls(A,b,10**-5)
end = time()
end - start # Vreme bez korišćenja initial support vektora

0.0020754337310791016

In [13]:
support = np.nonzero(x)[0]
start = time()
x_sup, res = fnnls(A,b,10**-5,P_initial = support)
end = time()
end - start # Vreme sa korišćenjem initial support vektora

0.0009930133819580078

Postoje i varijante FNNLS algoritma kao što su težinski FNNLS, gde se umesto matrica $A^TA$ i $A^Tb$ koriste $A^TVA$ i $A^TVb$, gde je $V$ dijagonalna matrica koja sadrži težine. Takođe, moguće je konstruisati i algoritam koji ne zahteva ne-negativnost svih komponenti rešenja, već samo nekog podskupa.